In [56]:
!pip install py4j

You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
import matplotlib.pyplot as plt 
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql.functions import udf
from pyspark.sql.functions import when
from pyspark import SparkContext 
from pyspark.sql.functions import col, split, ltrim, substring
import pyspark.sql as SQL
from pyspark.sql.functions import unix_timestamp, from_unixtime, date_format, \
        from_utc_timestamp, to_utc_timestamp, date_format, dayofmonth, monotonically_increasing_id, dayofyear
import datetime
import calendar
import pandas as pd
from pyspark.ml.linalg import Vectors
from pyspark.ml.regression import LinearRegression
from pyspark.sql.types import *
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from pyspark.sql import SQLContext as sqc
from pyspark import SparkConf, SparkContext, SQLContext
from pyspark.sql import *
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.linalg.distributed import RowMatrix
from pyspark import SparkConf, SparkContext
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.mllib.regression import LabeledPoint
from pyspark.ml.classification import LogisticRegression

from pyspark.sql.functions import col
from pyspark.ml import Pipeline
from pyspark.ml.regression import *
from pyspark.ml.feature import *
from pyspark.ml.evaluation import *
from pyspark.ml import *
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator


In [2]:
conf = (SparkConf()
         .setAppName("s3a_test")
         .set("spark.executor.instances", "8")
         .set("spark.executor.cores", 2)
         .set("spark.shuffle.compress", "true")
         .set("spark.io.compression.codec", "snappy")
         .set("spark.executor.memory", "2g"))

In [3]:
sc = SparkContext(conf = conf)

In [32]:
# descibed by heteogenous features 

# # header = 'https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2017-'

# # months = ['02.csv', '03.csv', '04.csv', '05.csv', '06.csv', '07.csv',\
# #           '08.csv', '09.csv', '10.csv', '11.csv', '12.csv']

# # all_taxi_months = spark.read.format("csv").load('https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2017-01.csv', header = True).cache()

# # for m in months:
# #     month = spark.read.format("csv").load((header + months), header = True).cache()
# #     all_taxi_months = all_taxi_months.union(month)

In [34]:
#jan_df.write.parquet("s3a://taxi-data-545/s3.parquet")

In [40]:
# text = sc.textFile("s3a://taxi-data-545/untitled.txt")

In [42]:
# parts = text.map(lambda l: l.split("\t"))


In [43]:
# from pyspark import SparkContext, SparkConf
# conf = SparkConf().setAppName("app-name-of-your-choice")
# sc = SparkContext(conf=conf)

PythonRDD[16] at RDD at PythonRDD.scala:48


In [5]:
# sqlContext = SQLContext(sc)
# #sqlContext.createDataFrame(pandas_df)
# sc._jsc.hadoopConfiguration().set("spark.sql.parquet.output.committer.class", "org.apache.spark.sql.parquet.DirectParquetOutputCommitter")
# sc._jsc.hadoopConfiguration().set("fs.s3.impl", "org.apache.hadoop.fs.s3native.NativeS3FileSystem")
# sc._jsc.hadoopConfiguration().set("fs.s3.impl", "org.apache.hadoop.fs.s3native.NativeS3FileSystem")



NameError: name 'SQLContext' is not defined

In [4]:
sqlContext = SQLContext(sc)

In [9]:
training_df = sqlContext.read.load("training.json")

In [ ]:
# training_df.select("label", "features").write.save("training.json", format="json")

In [8]:
training_df.select("*").write.save("training.json")

In [10]:
training_df.show(10)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|   10|(468,[0,1,2,8,10,...|
|    4|(468,[0,1,2,4,6,8...|
|    9|(468,[0,1,2,4,6,8...|
|    6|(468,[0,1,2,4,6,8...|
|    7|(468,[0,1,2,4,5,6...|
|    7|(468,[0,1,2,4,5,6...|
|    5|(468,[0,1,2,4,5,6...|
|    6|(468,[0,1,2,4,5,6...|
|    9|(468,[0,1,2,4,5,6...|
|   13|(468,[0,1,2,4,5,6...|
+-----+--------------------+
only showing top 10 rows



## Decision Tree Classifier

In [14]:


# Load the data stored in LIBSVM format as a DataFrame.
data = training_df.sample(False,0.01,None).cache()

# Index labels, adding metadata to the label column.
# Fit on whole dataset to include all labels in index.
labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(data)
# # Automatically identify categorical features, and index them.
# # We specify maxCategories so features with > 4 distinct values are treated as continuous.
featureIndexer =\
     VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(data)

# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = data.randomSplit([0.7, 0.3])

# Train a DecisionTree model.
dt = DecisionTreeClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures")

# Chain indexers and tree in a Pipeline
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, dt])

# Train model.  This also runs the indexers.
model = pipeline.fit(trainingData)

# Make predictions.
predictions = model.transform(testData)

# Select example rows to display.
predictions.select("prediction", "indexedLabel", "features").show(5)

# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g " % (1.0 - accuracy))

treeModel = model.stages[2]
# summary only
print(treeModel)

+----------+------------+--------------------+
|prediction|indexedLabel|            features|
+----------+------------+--------------------+
|       6.0|         6.0|(468,[0,1,2,4,5,6...|
|       6.0|         6.0|(468,[0,1,2,4,5,6...|
|       6.0|         6.0|(468,[0,1,2,4,5,6...|
|       6.0|         6.0|(468,[0,1,2,4,5,6...|
|       6.0|         6.0|(468,[0,1,2,4,5,6...|
+----------+------------+--------------------+
only showing top 5 rows

Test Error = 0.0183442 
DecisionTreeClassificationModel (uid=DecisionTreeClassifier_485999e72cdf76bc588b) of depth 5 with 21 nodes


## MultiLayer Perceptron 

In [32]:
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Load training data
data1 = training_df.sample(False,0.01,None).cache()

# numClasss = data1.select('')

# Split the data into train and test
splits = data1.randomSplit([0.6, 0.4], 1234)
train = splits[0]
test = splits[1]

# specify layers for the neural network:
# input layer of size 4 (features), two intermediate of size 5 and 4
# and output of size 3 (classes)
layers = [468, 500, 150, 1]

# create the trainer and set its parameters
trainer = MultilayerPerceptronClassifier(maxIter=10, layers=layers, blockSize=128, seed=1234)

# train the model
model = trainer.fit(train)

# compute accuracy on the test set
result = model.transform(test)
predictionAndLabels = result.select("prediction", "label")
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
print("Test set accuracy = " + str(evaluator.evaluate(predictionAndLabels)))

Py4JJavaError: An error occurred while calling o1206.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 95.0 failed 1 times, most recent failure: Lost task 1.0 in stage 95.0 (TID 265, localhost, executor driver): java.lang.ArrayIndexOutOfBoundsException

Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.org$apache$spark$scheduler$DAGScheduler$$failJobAndIndependentStages(DAGScheduler.scala:1499)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1487)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1486)
	at scala.collection.mutable.ResizableArray$class.foreach(ResizableArray.scala:59)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:48)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:1486)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$handleTaskSetFailed$1.apply(DAGScheduler.scala:814)
	at org.apache.s

In [23]:
data.show(10)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|    9|(468,[0,1,2,4,5,8...|
|   10|(468,[0,1,2,4,5,8...|
|    6|(468,[0,1,2,4,5,8...|
|   13|(468,[0,1,2,4,5,8...|
|    8|(468,[0,1,2,4,5,8...|
|    9|(468,[0,1,2,4,8,1...|
|   10|(468,[0,1,2,4,5,6...|
|    3|(468,[0,1,2,4,5,8...|
|   11|(468,[0,1,2,4,5,7...|
|    1|(468,[0,1,2,4,5,7...|
+-----+--------------------+
only showing top 10 rows



In [24]:
labeledPointRDD = training_df.select('*').rdd.map(lambda row:  LabeledPoint(row.label, row.features))

In [26]:
labeledPointRDD

DataFrame[label: double, features: vector]

In [29]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator

# Load the data stored in LIBSVM format as a DataFrame.
data = labeledPointRDD
# Automatically identify categorical features, and index them.
# We specify maxCategories so features with > 4 distinct values are treated as continuous.
featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(data)

# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = data.randomSplit([0.7, 0.3])

# Train a DecisionTree model.
dt = DecisionTreeRegressor(featuresCol="indexedFeatures")

# Chain indexer and tree in a Pipeline
pipeline = Pipeline(stages=[featureIndexer, dt])

# Train model.  This also runs the indexer.
model = pipeline.fit(trainingData)

# Make predictions.
predictions = model.transform(testData)

# Select example rows to display.
predictions.select("prediction", "label", "features").show(5)

# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

treeModel = model.stages[1]
# summary only
print(treeModel)

+------------------+-----+--------------------+
|        prediction|label|            features|
+------------------+-----+--------------------+
| 149.7002092245971|  1.0|(743,[0,1,2,3,4,5...|
| 149.7002092245971|  1.0|(743,[0,1,2,3,4,5...|
|161.35843801280842|  1.0|(743,[0,1,2,4,5,6...|
|161.35843801280842|  1.0|(743,[0,1,2,4,5,7...|
|161.35843801280842|  1.0|(743,[0,1,2,4,5,8...|
+------------------+-----+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 68.8796
DecisionTreeRegressionModel (uid=DecisionTreeRegressor_4ddeab5f28ce4b1c41b8) of depth 5 with 63 nodes


In [31]:
# Select example rows to display.
predictions.select("prediction", "label", "features").show(1000)

+------------------+-----+--------------------+
|        prediction|label|            features|
+------------------+-----+--------------------+
| 149.7002092245971|  1.0|(743,[0,1,2,3,4,5...|
| 149.7002092245971|  1.0|(743,[0,1,2,3,4,5...|
|161.35843801280842|  1.0|(743,[0,1,2,4,5,6...|
|161.35843801280842|  1.0|(743,[0,1,2,4,5,7...|
|161.35843801280842|  1.0|(743,[0,1,2,4,5,8...|
| 149.7002092245971|  1.0|(743,[0,1,2,4,5,9...|
|161.35843801280842|  1.0|(743,[0,1,2,4,5,9...|
| 149.7002092245971|  1.0|(743,[0,1,2,4,9,1...|
| 149.7002092245971|  3.0|(743,[0,1,2,3,4,5...|
|  149.278479946783|  3.0|(743,[0,1,2,5,8,1...|
| 149.7002092245971|  4.0|(743,[0,1,2,3,4,5...|
| 149.7002092245971|  4.0|(743,[0,1,2,3,4,5...|
| 149.7002092245971|  4.0|(743,[0,1,2,3,4,5...|
| 149.7002092245971|  4.0|(743,[0,1,2,3,4,5...|
| 149.7002092245971|  4.0|(743,[0,1,2,3,4,5...|
| 149.7002092245971|  4.0|(743,[0,1,2,3,4,5...|
| 149.7002092245971|  4.0|(743,[0,1,2,3,4,5...|
| 149.7002092245971|  4.0|(743,[0,1,2,3,

In [33]:
predictions.select("features").show(1000)

+--------------------+
|            features|
+--------------------+
|(743,[0,1,2,3,4,5...|
+--------------------+



AttributeError: 'NoneType' object has no attribute 'collect'

## Random Forest Classifier

In [34]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Load and parse the data file, converting it to a DataFrame.
data= training_df#.sample(False,,None).cache()
# Index labels, adding metadata to the label column.
# Fit on whole dataset to include all labels in index.
labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(data)

# # Automatically identify categorical features, and index them.
# # Set maxCategories so features with > 4 distinct values are treated as continuous.
featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(data)

# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = data.randomSplit([0.7, 0.3])

# Train a RandomForest model.
rf = RandomForestClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures", numTrees=25)

# Convert indexed labels back to original labels.
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel",
                               labels=labelIndexer.labels)

# Chain indexers and forest in a Pipeline
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, rf, labelConverter])

# Train model.  This also runs the indexers.
model = pipeline.fit(trainingData)

# Make predictions.
predictions = model.transform(testData)

# Select example rows to display.
predictions.select("predictedLabel", "label", "features").show(5)

# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy))

rfModel = model.stages[2]
print(rfModel)  # summary only

+--------------+-----+--------------------+
|predictedLabel|label|            features|
+--------------+-----+--------------------+
|            13|    1|(468,[0,1,2,4,5,6...|
|             1|    1|(468,[0,1,2,4,5,6...|
|             1|    1|(468,[0,1,2,4,5,6...|
|             1|    1|(468,[0,1,2,4,5,6...|
|             1|    1|(468,[0,1,2,4,5,6...|
+--------------+-----+--------------------+
only showing top 5 rows

Test Error = 0.625353
RandomForestClassificationModel (uid=RandomForestClassifier_4c02b595a923fb9f5c07) with 25 trees


## Logistic Regression

In [16]:
training= training_df.sample(False,0.01,None).cache()


lr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)

# Fit the model
lrModel = lr.fit(training)


# Print the coefficients and intercepts for logistic regression with multinomial family
print("Multinomial coefficients: " + str(mlrModel.coefficientMatrix))
print("Multinomial intercepts: " + str(mlrModel.interceptVector))

NameError: name 'mlrModel' is not defined

In [27]:
# Extract the summary from the returned LogisticRegressionModel instance trained
# in the earlier example
trainingSummary = lrModel.summary

# Obtain the objective per iteration
objectiveHistory = trainingSummary.objectiveHistory
print("objectiveHistory:")
for objective in objectiveHistory:
    print(objective)

# Obtain the receiver-operating characteristic as a dataframe and areaUnderROC.
trainingSummary.roc.show()
print("areaUnderROC: " + str(trainingSummary.areaUnderROC))

# Set the model threshold to maximize F-Measure
fMeasure = trainingSummary.fMeasureByThreshold
maxFMeasure = fMeasure.groupBy().max('F-Measure').select('max(F-Measure)').head()
bestThreshold = fMeasure.where(fMeasure['F-Measure'] == maxFMeasure['max(F-Measure)']) \
    .select('threshold').head()['threshold']
lr.setThreshold(bestThreshold)

RuntimeError: No training summary available for this LogisticRegressionModel